# adulib.utils

General utility functions.

In [ ]:
#|default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
def check_mutual_exclusivity(*args, check_falsy=True):
    """
    Check if only one of the arguments is falsy (or truthy, if check_falsy is False).
    """
    if check_falsy:
        return sum(bool(x) for x in args) == 1
    else:
        return sum(bool(x) for x in args) == 1

In [ ]:
import adulib.utils